<a href="https://colab.research.google.com/github/RybakovAlex/Python-for-Analythics/blob/main/%D0%91%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82_%22GB_Homework_5_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Урок 5. Статистика в работе аналитика с библиотекой Scipy\
Можно решить 2 задачи на выбор:

1. Взять датасет из домашнего задания №2. Проверить гипотезу о том, в каком варианте теста (control/personalization) больше конверсия (converted) и значимо ли это отличие статистически.

In [44]:
# Импортирую нужные библиотеки
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
import seaborn as sns

In [45]:
df = pd.read_csv('/content/marketing_campaign.csv', header = 0, sep = ',', index_col = None)
df.head()

,user_id,date_served,marketing_channel,variant,language_displayed,converted
0,a1000,01.01.18,House Ads,personalization,English,True
1,a1001,01.01.18,House Ads,personalization,English,True
2,a1002,01.01.18,House Ads,personalization,English,True
3,a1003,01.01.18,House Ads,personalization,English,True
4,a1004,01.01.18,House Ads,personalization,English,True


In [46]:
data = df.groupby('variant')['converted'].agg(['sum', 'count']).reset_index()
data = data.set_index('variant')
data['CR'] = data['sum'] / data['count']
data = data.rename(columns = {'sum': 'converted'})
data

,converted,count,CR
variant,,,
control,371,5076,0.073089
personalization,705,4946,0.142539


In [47]:
print(f"Конверсия в варианте control - {data.loc['control', 'CR']: .0%}, конверсия в варианте personalization - {data.loc['personalization', 'CR']: .0%}")

Конверсия в варианте control -  7%, конверсия в варианте personalization -  14%


In [48]:
data['lost'] = data['count'] - data['converted']
data = data.drop(['count', 'CR'], axis=1)
data

,converted,lost
variant,,
control,371,4705
personalization,705,4241


In [49]:
result = stats.chi2_contingency(data, correction = False)
chisq, pvalue = result[:2]
print('chisq = {}, pvalue = {}'.format(chisq, pvalue))
print('\033[93m'+'Вывод: поскольку значимость А/В теста 2.9568009705199523e-29 менее допустимого уровня значимости 0,05,\nто нулевую гипотезу о равенстве конверсий в группах можно отклонить, т.е. вариант теста personalization,\nдействительно дает статистически большую конверсию, чем вариант теста control (CR в 2 раза больше).')

chisq = 126.07767827548582, pvalue = 2.9568009705199523e-29
Вывод: поскольку значимость А/В теста 2.9568009705199523e-29 менее допустимого уровня значимости 0,05,
то нулевую гипотезу о равенстве конверсий в группах можно отклонить, т.е. вариант теста personalization,
действительно дает статистически большую конверсию, чем вариант теста control (CR в 2 раза больше).


2. Цех может производить стулья и столы. На производство стула идет 5 единиц материала, на производство стола - 20 единиц (футов красного дерева). Стул требует 10 человеко-часов, стол - 15. Имеется 400 единиц материала и 450 человеко-часов. Прибыль при производстве стула - 45 долларов США, при производстве стола - 80 долларов США. Сколько надо сделать стульев и столов, чтобы получить максимальную прибыль?

In [52]:
from scipy.optimize import Bounds
from scipy.optimize import minimize
def value(x):
    return - 45*x[0] - 80*x[1]

In [84]:
neq_cons = {'type': 'ineq',
             'fun': lambda x: np.array ([400 - 5 * x [0] - 20 * x [1] ,
                                         450 - 10 * x [0] - 15 * x [1]])}
bnds = Bounds([0, 0], [np.inf, np.inf])
x0 = np.array([10, 10])
res = minimize(value, x0, method='SLSQP', constraints=neq_cons, bounds=bnds)
res.x
print(f'Мах прибыль: {-value([24, 14])}. Производство стульев: {round(res.x[0], 0)} Производство столов: {round(res.x[1], 0)}')
print(f'\033[93m'+'Вывод: максимальная прибыль: 2200 USD достигается при производстве 24 стульев и 14 столов.')

Мах прибыль: 2200. Производство стульев: 24.0 Производство столов: 14.0
Вывод: максимальная прибыль: 2200 USD достигается при производстве 24 стульев и 14 столов.


3. Средний накопленный доход на пользователя с сайта (cumulative ARPU) составил: day1 0.4$, day3 0.6$, day7 0.8$, day14 1.10$ , day21 1.30$, day30 1.40$. Подберите аппроксимирующую кривую и сделайте прогноз, сколько в среднем принесет денег пользователь на 90й день. Обоснуйте выбор одной из аппроксимирующей кривой вида: y = a*b^x или y = a*ln(x) + b.